In [ ]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import numpy as np
from numpy import mean
import json
import gzip
from subprocess import check_output
import math
import ast
%matplotlib inline

In [ ]:
production_df = pd.read_excel('../data/tmdb_production_data.xlsx', index_col=0)
movie_df = pd.read_excel('../data/tmdb_movie_data.xlsx', index_col=0)
genre_df = pd.read_excel('../data/movie_genres_data.xlsx', index_col=0)
cast_df = pd.read_excel('../data/tmdb_cast_data.xlsx', index_col=0)

In [ ]:
director_df = cast_df.loc[cast_df['cast_known_for_department']=='Directing']
director_df

In [ ]:
director_pop = director_df[['movie_id', 'cast_name', 'cast_popularity']]
director_pop = director_pop.drop(columns={'movie_id'})
director_pop = director_pop.groupby('cast_name').mean()
director_pop = director_pop.sort_values(by=['cast_popularity'], ascending=False)
director_pop.reset_index(inplace=True)
director_pop = director_pop.rename(columns={'cast_name':'Director', 'cast_popularity':"Director's Popularity"})
director_pop

In [ ]:
director_movie = director_df[['movie_id', 'cast_name']]
director_movie = director_movie.groupby("cast_name").agg({"movie_id": lambda x: list(x)})
director_movie.reset_index(inplace=True)
director_movie = director_movie.rename(columns={'cast_name':'Director'})
director_movie

In [ ]:
test = director_movie['movie_id'][457][0]
test

In [ ]:
movie_popularity[movie_popularity['movie_id'] == test]

In [ ]:
pop_data = production_df[['movie_title', 'movie_revenue']]
movie_id = movie_df[['movie_title', 'movie_id']]
movie_popularity = pop_data.merge(movie_id)
movie_popularity = movie_popularity.drop(columns={'movie_title'})
movie_popularity

In [ ]:
repeated_rows = []
for movie_id, movie_revenue in zip(movie_popularity['movie_id'], movie_popularity['movie_revenue']):
    directors = director_movie[director_movie['movie_id'].apply(lambda x: movie_id in x)]['Director'].values
    for director in directors:
        repeated_rows.append({'movie_id': movie_id, 'movie_revenue': movie_revenue, 'Director': director})

movie_popularity_repeated = pd.DataFrame(repeated_rows)

In [ ]:
print(movie_popularity_repeated)

In [ ]:
director_popularity_repeated = pd.DataFrame(movie_popularity_repeated)
director_popularity_repeated = director_popularity_repeated.loc[director_popularity_repeated['movie_revenue'] > 0]
director_popularity_repeated

In [ ]:
director_merged = director_popularity_repeated.merge(director_pop)
director_merged = director_merged.drop(columns={'movie_id'})
director_merged

In [ ]:
pop_count = director_merged[["Director's Popularity"]]
pop_count["Director's Popularity"] = round(pop_count["Director's Popularity"],0)
pop_count = pop_count.groupby("Director's Popularity").count()
pop_count

In [ ]:
direct_data = director_merged.groupby('Director').mean()
direct_data = direct_data.rename(columns={'movie_revenue':'Revenue (Millions)'})
direct_data = direct_data.sort_values(by=["Director's Popularity"], ascending=False)
direct_data["Director's Popularity"] = round(direct_data["Director's Popularity"],0)
direct_data = direct_data.reset_index()
direct_data = direct_data.drop(columns={'Director'})
direct_data = direct_data.rename(columns={"Director's Popularity":'Directors Popularity'})
direct_data = direct_data.groupby("Directors Popularity").mean()
direct_data = direct_data.reset_index()
direct_data['Revenue (Millions)'] = round(direct_data['Revenue (Millions)']/1000000,0)
direct_data

In [ ]:
direct_data

In [ ]:
binned_popularity = pd.cut(direct_data['Directors Popularity'], bins=5)

In [ ]:
grouped_df = direct_data.groupby(binned_popularity)['Revenue (Millions)'].mean().reset_index()
grouped_df

In [ ]:
plt.figure(figsize=(14,11))
sns.set(style="ticks", context="talk")
plt.style.use("dark_background")


bar = sns.barplot(x='Directors Popularity', y='Revenue (Millions)', data=direct_data, color='#F9C31A', ci=False);

for x in range(len(means)):
    bar.text(x,means[x]+2.5, str(round(means[x],0)),
            fontdict = dict(color='white', fontsize=18),
            horizontalalignment = 'center')